# VWorld API를 활용한 2D/3D 지도 시각화

## 사하구 하단동 병원 목록을 마커로 표시한 2D지도 시각화

folium 라이브러리 설치

In [1]:
pip install folium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


병원 주소를 위도/경도로 변환하는 함수 정의

In [2]:
import requests

def vworld_geocode(address, api_key):
    url = "http://api.vworld.kr/req/address?"
    params = {
        "service": "address",
        "request": "getcoord",
        "format": "json",
        "crs": "EPSG:4326",
        "type": "road",
        "key": api_key,
        "address": address
    }
    res = requests.get(url, params=params)
    if res.status_code == 200:
        result = res.json()
        if result["response"]["status"] == "OK":
            point = result["response"]["result"]["point"]
            return float(point["y"]), float(point["x"])
    return None, None

CSV 병원 목록을 불러와 주소 변환 및 저장

In [5]:
import pandas as pd
import time

df = pd.read_csv("병원목록.csv", encoding="cp949")

api_key = "38B9DFC6-0193-3B7A-B692-EF484D2910E7" 

latitudes = []
longitudes = []

for addr in df["의료기관주소(도로명)"]:
    lat, lon = vworld_geocode(addr, api_key)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(0.2)

df["lat"] = latitudes
df["lon"] = longitudes

VWorld 배경지도를 사용한 지도 생성 및 병원 위치 마커 표시

In [13]:
import folium 

tile_url = f"https://api.vworld.kr/req/wmts/1.0.0/{api_key}/Base/{{z}}/{{y}}/{{x}}.png"

# 부산 하단동 중심
m = folium.Map(location=[35.1065, 128.9715], zoom_start=15, tiles=None)

folium.TileLayer(
    tiles=tile_url,
    attr="VWorld",
    name="VWorld Base",
    overlay=False,
    control=True
).add_to(m)

for _, row in df.iterrows():
    if pd.notnull(row["lat"]) and pd.notnull(row["lon"]):
        folium.Marker(
            location=[row["lat"], row["lon"]],
            popup=f"{row['의료기관명']}<br>{row['의료기관주소(도로명)']}",
            tooltip=row["의료기관명"],
            icon=folium.Icon(color="red", icon="plus-sign", prefix="fa")
        ).add_to(m)

folium.LayerControl().add_to(m)


m.save("hadan_hospital_map.html")

m

## 사하구 하단동 일대 CCTV 위치를 WebGL 기반 3D지도로 시각화

pydeck 라이브러리 설치

In [7]:
pip install pydeck

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VWorld API를 사용하여 CCTV 위치 정보 요청

In [ ]:
params = {
    'service': 'data',
    'request': 'GetFeature',
    'key': api_key,
    'data': 'LT_P_UTISCCTV',
    'format': 'json',
    'geometry': 'true',
    'size': 1000,
    'geomFilter': 'BOX(128.940,35.060,128.970,35.085)'  # 하단동 영역
}

res = requests.get("http://api.vworld.kr/req/data", params=params)
geojson_data = res.json()
features = geojson_data["response"]["result"]["featureCollection"]["features"]

CCTV 정보를 3D ColumnLayer 형태로 구성

In [9]:
cctv_data = []

for f in features:
    lon, lat = f["geometry"]["coordinates"]
    props = f["properties"]
    cctv_data.append({
        "lon": lon,
        "lat": lat,
        "height": 100,
        "name": props.get("cctvname", "CCTV")
    })

pydeck을 이용한 시각화 및 HTML로 저장

In [12]:
import pydeck as pdk

layer = pdk.Layer(
    "ColumnLayer",
    cctv_data,
    get_position=["lon", "lat"],
    get_elevation="height",
    elevation_scale=1,
    radius=50,
    get_fill_color="[200, 30, 0, 160]",
    pickable=True,
    auto_highlight=True,
)

view_state = pdk.ViewState(
    latitude=35.072,
    longitude=128.954,
    zoom=14,
    pitch=45
)

deck = pdk.Deck(
    layers=[layer],
    initial_view_state=view_state,
    tooltip={"text": "CCTV: {name}"},
    height=600
)

deck.to_html("hadan_cctv_3d.html", notebook_display=False)

deck

{
  "initialViewState": {
    "latitude": 35.072,
    "longitude": 128.954,
    "pitch": 45,
    "zoom": 14
  },
  "layers": [
    {
      "@@type": "ColumnLayer",
      "autoHighlight": true,
      "data": [
        {
          "height": 100,
          "lat": 35.07994,
          "lon": 128.96851,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08026,
          "lon": 128.96837,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08026,
          "lon": 128.96837,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08217,
          "lon": 128.96599,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08052,
          "lon": 128.95295,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08117,
          "lon": 128.95544,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07889,
          "lon": 128.96566,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08004,
          "lon": 128.96666,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07717,
          "lon": 128.96261,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07717,
          "lon": 128.96261,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07704,
          "lon": 128.96297,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08392,
          "lon": 128.96883,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08392,
          "lon": 128.96883,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07894,
          "lon": 128.96475,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07894,
          "lon": 128.96475,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07894,
          "lon": 128.96475,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07894,
          "lon": 128.96475,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.07955,
          "lon": 128.9648,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08123,
          "lon": 128.96712,
          "name": "\ubd80\uc0b0\uad11\uc5ed\uc2dc \uc0ac\ud558\uad6c\uccad \ub2e4\ubaa9\uc801"
        },
        {
          "height": 100,
          "lat": 35.08123,
      